In [1]:
import numpy as np

# Ocena klasyfikacji binarnej:

**Accuracy** (Dokładność) to najprostsza miara jakości modelu, która mierzy, jaki procent wszystkich przewidywań był poprawny. Jest to **stosunek prawidłowo sklasyfikowanych** przykładów **do wszystkich** przykładów w zbiorze testowym.

Najlepiej aby klasy były zrównoważone, inaczej accuracy może być nieodpowiednią miarą. Przykładowo, jeśli w jednej klasie jest 90 % przypadków, to accuracy może być wysokie, pomimo tego, że model źle klasyfikuje drugą klasę. 

In [10]:
def accuracy(correct, predicted):
    return (correct == predicted).sum() / correct.size

**Precision** (Precyzja) mierzy, jak dokładnie model klasyfikuje pozytywne przypadki, czyli jaka część przykładów przewidzianych jako pozytywne (1) faktycznie była pozytywna. Jest to **stosunek poprawnie przewidzianych pozytywnych** etykiet **do wszystkich pozytywnie przewidzianych**, w tym niepoprawnie.

Może się okazać ważna przykładowo w przypadku klasyfikacji chorób, wtedy chcemy żeby ta precyzja była jak największa. Model przewidywałby poprawnie chorobę z prawdopodobieństwem równym miary precision.

In [3]:
def precision(correct, predicted):
    tp = np.sum((correct == 1) & (predicted == 1))
    fp = np.sum((correct == 0) & (predicted == 1))
    return tp / (tp + fp) if (tp + fp) != 0 else 0

**Recall** (Czułość, Wykrywalność) mierzy, jak wiele rzeczywistych pozytywnych przypadków model poprawnie wykrył. Jest to **stosunek liczby prawdziwych pozytywów do liczby wszystkich rzeczywistych pozytywnych** przykładów.

W przypadkach, gdy pominięcie pozytywnych przypadków jest kosztowne (np. wykrywanie oszustw – lepiej wykryć więcej oszustw, nawet kosztem fałszywych alarmów).

In [4]:
def recall(correct, predicted):
    tp = np.sum((correct == 1) & (predicted == 1))
    fn = np.sum((correct == 1) & (predicted == 0))
    return tp / (tp + fn) if (tp + fn) != 0 else 0

Niech przykładowy model klasyfikuje e-maile jako spam (klasa 1) lub nie spam (klasa 0).

-> Scenariusz 1: Wysoka Precision, Niski Recall<br>
Model bardzo ostrożnie oznacza e-mail jako spam.<br>
Oznacza jako spam tylko wtedy, gdy jest pewien.<br>
Konsekwencja: Przegapi wiele spamów, ale rzadko myli zwykłe e-maile ze spamem.

-> Scenariusz 2: Wysoki Recall, Niska Precision<br>
Model oznacza wszystkie podejrzane e-maile jako spam.<br>
Konsekwencja: Wykrywa prawie cały spam, ale często oznacza zwykłe e-maile jako spam.<br>

**F1-score** to średnia harmoniczna precyzji i recall, która daje dobry balans pomiędzy tymi dwiema miarami. F1-score jest szczególnie użyteczny, gdy dane są niezbalansowane (np. kiedy jedna klasa występuje znacznie częściej niż druga).

F1-score używany jest, gdy precision i recall są równie ważne, ale jeśli chcesz podkreślić znaczenie jednej z tych metryk, używasz F_beta-score.

-> beta = 1 <- klasyczny F1-score (równoważy precision i recall)<br>
-> beta > 1 <- większy nacisk na recall<br>
-> beta < 1 <- większy nacisk na precision<br>

In [5]:
def f_beta_score(correct, predicted, beta = 1):
    prec = precision(correct, predicted)
    rec = recall(correct, predicted)
    return (1 + beta**2) * (prec * rec) / (beta**2 * prec + rec) if (beta**2 * prec + rec) != 0 else 0

# Perceptron:

In [6]:
class Perceptron:
    def __init__(self, num_features, epochs = 500, learning_rate=0.01, epsilon=1e-10):
        self.weights = np.random.rand(num_features)
        self.bias = np.random.rand(1)[0]
        self.epochs = epochs
        self.epsilon = epsilon
        self.learning_rate = learning_rate

    def predict(self, x): # funkcja aktywacji
        return np.where(np.dot(x, self.weights) + self.bias >= 0, 1, 0)
    
    def train(self, x_train, y_train):
        for epoch in range(self.epochs):
            prev_weights = self.weights.copy()

            for x, y in zip(x_train, y_train):
                prediction = self.predict(x)
                update = y - prediction  # -1, 0, lub 1; jeśli 0 to wagi się nie aktualizują, ponieważ była poprawna predykcja
                
                self.weights += self.learning_rate * update * x
                self.bias += self.learning_rate * update

            # Sprawdzenie, czy wagi się stabilizują
            if np.linalg.norm(self.weights - prev_weights) < self.epsilon:
                return epoch + 1
        return self.epochs

    def test(self, X):
        return self.predict(X)

In [7]:
import csv    
def open_csv(file_path):
    x, y = [], []
    with open(file_path, 'r') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=' ')
        for row in csvreader:
            x.append(list(map(float, row[:-1])))
            y.append(int(row[-1]))
    return np.array(x), np.array(y)

x, y = open_csv('dataset.csv')       
print(x)
print(y)
x_train = x[:7000]
y_train = y[:7000]

x_test = x[7000:]
y_test = y[7000:]

[[15.73094725  7.18528284 64.41038317 24.66428237]
 [83.44155279 63.01982687 75.83614336 67.61127283]
 [99.39100517  9.61315971 14.93347999 72.15498409]
 ...
 [87.97905359 25.80428991 22.24284486 88.04449276]
 [80.20238287 21.64052602 41.48732363  3.6147353 ]
 [ 0.62492591 99.65803104 81.97150773 77.52295713]]
[0 1 1 ... 1 0 0]


In [8]:
perceptron = Perceptron(4)
epochs_used = perceptron.train(x_train, y_train)

print(f"Trening zakończony po {epochs_used} epokach.")

predicted = perceptron.test(x_test)
print(predicted)

Trening zakończony po 500 epokach.
[1 1 1 ... 1 0 1]


In [11]:
print(f"Accuracy: {accuracy(y_test, predicted)}")
print(f"Precision: {precision(y_test, predicted)}")
print(f"Recall: {recall(y_test, predicted)}")
print(f"F1 Score: {f_beta_score(y_test, predicted)}")

Accuracy: 0.8676666666666667
Precision: 0.7660332541567696
Recall: 0.9976798143851509
F1 Score: 0.86664427275781
